<h1>Elecciones 2023</h1>

<h2>Configuracion</h2>

<b>Instalations</b>

In [1]:
%pip install --upgrade plotly
%pip install pandas
%pip install numpy
%pip install seaborn
%pip install matplotlib
%pip install scipy
%pip install geopandas


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\marco\OneDrive\Documents\Marcos\Programacion\PYTHON\elecciones\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\marco\OneDrive\Documents\Marcos\Programacion\PYTHON\elecciones\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\marco\OneDrive\Documents\Marcos\Programacion\PYTHON\elecciones\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\marco\OneDrive\Documents\Marcos\Programacion\PYTHON\elecciones\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\marco\OneDrive\Documents\Marcos\Programacion\PYTHON\elecciones\venv\Scripts\python.exe -m pip install --upgrade pip' command.


<b>Imports</b>

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import skewnorm
import geopandas as gpd

<b>Pandas Config</b>

In [2]:
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)

<h2>Base de datos</h2>

<b>Importar BD General</b>

In [23]:
file_name = "bases/ResultadosElectorales_2023.csv"
df_data = pd.read_csv(file_name, low_memory=False, encoding="utf-8")
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5868102 entries, 0 to 5868101
Data columns (total 23 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   año                       int64  
 1   eleccion_tipo             object 
 2   recuento_tipo             object 
 3   padron_tipo               object 
 4   distrito_id               int64  
 5   distrito_nombre           object 
 6   seccionprovincial_id      int64  
 7   seccionprovincial_nombre  object 
 8   seccion_id                int64  
 9   seccion_nombre            object 
 10  circuito_id               object 
 11  circuito_nombre           object 
 12  mesa_id                   int64  
 13  mesa_tipo                 object 
 14  mesa_electores            int64  
 15  cargo_id                  int64  
 16  cargo_nombre              object 
 17  agrupacion_id             int64  
 18  agrupacion_nombre         object 
 19  lista_numero              float64
 20  lista_nombre            

<b>Importar BD Mapeo Nomenclator</b>

In [24]:
# Fuente: la cree yo a partir de la web oficial resultados.gob.ar
file_name_nomenclator = "mesa_id_nomenclator.csv"
df_nomenclator = pd.read_csv(file_name_nomenclator, low_memory=False, encoding="utf-8")
# Renombrar las columnas
df_nomenclator.rename(columns={'i': 'nomenclator_id'}, inplace=True)
df_nomenclator.rename(columns={'mesa_id': 'nomenclator_mesa_id'}, inplace=True)
df_nomenclator.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104520 entries, 0 to 104519
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   nomenclator_mesa_id  104520 non-null  object
 1   nomenclator_id       104520 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.6+ MB


<b>Listar los cargos</b>

In [4]:
df_data["cargo_nombre"].unique()

array(['PRESIDENTE Y VICE', 'SENADOR NACIONAL', 'JEFE/A DE GOBIERNO',
       'DIPUTADO NACIONAL', 'DIPUTADO PROVINCIAL', 'SENADOR PROVINCIAL',
       'GOBERNADOR Y VICE', 'INTENDENTE', 'PARLAMENTO MERCOSUR NACIONAL',
       'PARLAMENTO MERCOSUR REGIONAL', 'CONCEJAL',
       'MIEMBROS DE JUNTA COMUNAL', 'DIPUTADOS/AS DE LA CIUDAD AUTONOMA'],
      dtype=object)

<b>Listar Tipos de votos</b>

In [6]:
df_data["votos_tipo"].unique()

array(['NULO', 'IMPUGNADO', 'RECURRIDO', 'COMANDO', 'POSITIVO',
       'EN BLANCO'], dtype=object)

<b>Preparar la informacion</b>

In [53]:
cargo_presidencial = "PRESIDENTE Y VICE"

# Agregar un ID combinado para asegurarase que sea unico
df_data['id_mesa_unico'] = df_data['distrito_id'].astype(str) + "-" + \
                          df_data['seccionprovincial_id'].astype(str) + "-" + \
                          df_data['seccion_id'].astype(str) + "-" + \
                          df_data['circuito_id'].astype(str) + "-" + \
                          df_data['mesa_id'].astype(str)

df_data['mesa_id_web'] = (
    df_data['distrito_id'].apply(lambda x: f'{x:02}') + 
    df_data['seccion_id'].apply(lambda x: f'{x:03}') + 
    df_data['mesa_id'].apply(lambda x: f'{x:05}') + 
    "X"
)

# Considerar solo los votos positivos, nulos o en blanco
selected_values = ["POSITIVO", "NULO", "EN BLANCO"]
votos_considerados = df_data[df_data['votos_tipo'].isin(selected_values)]

# Considerar solo los votos para presidente
votos_considerados = votos_considerados[votos_considerados["cargo_nombre"]==cargo_presidencial]

# Agregar el ID del nomenclator
# Merge df_data con df_nomenclator
votos_considerados = pd.merge(votos_considerados, df_nomenclator, left_on='mesa_id_web', right_on='nomenclator_mesa_id', how='left')
# Quitar nomenclator_mesa_id ya que va a estar duplicado
votos_considerados.drop(columns=['nomenclator_mesa_id'], inplace=True)

# Construir el link a la mesa en la pagina oficial del gobierno
votos_considerados["mesa_URL"] = (
    "https://resultados.gob.ar/elecciones/1/" +
    votos_considerados['nomenclator_id'].astype(str) +
    "/1/-1/-1/" +
    votos_considerados['distrito_nombre'].apply(lambda x: x.replace(" ", "-")) + "/" +
    votos_considerados['seccion_nombre'].apply(lambda x: x.replace(" ", "-")) + "/" +
    votos_considerados['circuito_nombre'].apply(lambda x: x.replace(" ", "-")) + "/" +
    votos_considerados['mesa_id_web']  
)



<b>Aplanar la BD</b>

In [87]:
## Re-estructurar el DF de forma que cada agrupacion (y votos en blanco y nulos) tiene su propia columna

# Crear nuevos DF dependiendo de 'votos_tipo'
positivos_df = votos_considerados[votos_considerados['votos_tipo'] == 'POSITIVO'].copy()
no_positivos_df = votos_considerados[votos_considerados['votos_tipo'] != 'POSITIVO'].copy()

# Crear las nuevas columnas por partidos con votos y el id del partido
positivos_df['new_col_name'] = positivos_df['agrupacion_nombre'].str.lower().str.replace(' ', '_')
positivos_df['new_col_name_id'] = positivos_df['new_col_name'] + '_id'
positivos_df.set_index(['mesa_id_web', 'new_col_name', 'new_col_name_id'], inplace=True)
# Pivotear la tabla para que los valores de new_col_name pasen a ser las columnas
positivos_pivot = positivos_df.pivot_table(index='mesa_id_web', 
                                           columns=['new_col_name', 'new_col_name_id'], 
                                           values=['votos_cantidad', 'agrupacion_id'], 
                                           aggfunc='first')

# Crear las nuevas columnas para los votos no positivos
no_positivos_df['new_col_name'] = no_positivos_df['votos_tipo'].str.lower().str.replace(' ', '_')
# Pivotear la tabla para que los valores de new_col_name pasen a ser las columnas
no_positivos_pivot = no_positivos_df.pivot_table(index='mesa_id_web', 
                                                 columns='new_col_name', 
                                                 values='votos_cantidad', 
                                                 aggfunc='sum')

# Aplanar los votos_considerados_flat y renombrar
new_columns = []
for col in positivos_pivot.columns:
    if col[0] == 'votos_cantidad':
        new_columns.append(col[1])  # Solo el partido por ahora
    elif col[0] == 'agrupacion_id':
        new_columns.append(f"{col[1]}_id")  # Partido con sufijo "_id"

positivos_pivot.columns = new_columns
no_positivos_pivot.columns = no_positivos_pivot.columns.get_level_values(0)

# Unir los DF pivoteads
merged_df = pd.merge(positivos_pivot, no_positivos_pivot, on='mesa_id_web', how='outer')

# Agregar las otras columnas (tomando el primer valor, ya que es constante para la mesa)
other_cols = votos_considerados.drop(columns=['votos_cantidad', 'agrupacion_nombre', 'agrupacion_id', 'votos_tipo'])
first_values = other_cols.groupby('mesa_id_web').first().reset_index()

# Union final
votos_considerados_flat = pd.merge(merged_df, first_values, on='mesa_id_web', how='left')

# Agregar total y porcentajes
columnas_con_votos = [
    'frente_de_izquierda_y_de_trabajadores_-_unidad', 
    'hacemos_por_nuestro_pais',
    'juntos_por_el_cambio',
    'la_libertad_avanza',
    'union_por_la_patria',
    'en_blanco',
    'nulo'
]

# Total
votos_considerados_flat['total_votos'] = votos_considerados_flat[columnas_con_votos].sum(axis=1)
votos_considerados_flat['total_positivos'] = votos_considerados_flat['total_votos'] - votos_considerados_flat['nulo'] - votos_considerados_flat['en_blanco']

# Porcentajes
for column in columnas_con_votos:
    perc_column = column + '_perc'
    votos_considerados_flat[perc_column] = (votos_considerados_flat[column] / votos_considerados_flat['total_positivos']) * 100

# Convertir "Nan" en 0 (por si hubo alguna divion por 0)
votos_considerados_flat.fillna(0, inplace=True)


# Mapeo de nombres completos de partidos a nombres cortos
partidos_columnas_map = {
    'frente_de_izquierda_y_de_trabajadores_-_unidad': 'fit', 
    'hacemos_por_nuestro_pais': 'hxp',
    'juntos_por_el_cambio': 'jxc',
    'la_libertad_avanza': 'lla',
    'union_por_la_patria': 'up'
}

# Agregar las columnas para el nombre completo de cada partido en mayúsculas
for full_name, short_name in partidos_columnas_map.items():
    uppercase_name = full_name.replace("_", " ").upper()
    votos_considerados_flat[short_name + '_nombre'] = uppercase_name

# Lista para establecer el orden de las columnas
ordered_columns = ['mesa_id_web', 'mesa_URL']

# Agregar el nombre, ID, votos y % de cada partido al orden de las columnas
for full_name, short_name in partidos_columnas_map.items():
    ordered_columns.extend([
        short_name + '_nombre', short_name + '_id', short_name + '_votos', short_name + '_perc'
    ])

# Renombrar las columnas de los partidos en el DataFrame
for full_name, short_name in partidos_columnas_map.items():
    votos_considerados_flat.rename(
        columns={
            full_name + '_id': short_name + '_id',
            full_name: short_name + '_votos',
            full_name + '_perc': short_name + '_perc'
        },
        inplace=True
    )

# Agregar columnas de votos no positivos
non_positivo_columns = ['en_blanco', 'en_blanco_perc', 'nulo', 'nulo_perc', 'total_positivos','total_votos']
ordered_columns.extend(non_positivo_columns)

# Agregar otras columnas (excluyendo aquellas ya incluidas)
other_columns = [col for col in votos_considerados_flat.columns if col not in ordered_columns]
ordered_columns.extend(other_columns)

# Reordenar el DataFrame
votos_considerados_flat = votos_considerados_flat[ordered_columns]


In [88]:
votos_considerados_flat.loc[2]



mesa_id_web                                                       0100100003X
mesa_URL                    https://resultados.gob.ar/elecciones/1/23956/1...
fit_nombre                     FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD
fit_id                                                                    136
fit_votos                                                                   6
fit_perc                                                                 2.64
hxp_nombre                                           HACEMOS POR NUESTRO PAIS
hxp_id                                                                    133
hxp_votos                                                                   5
hxp_perc                                                                 2.20
jxc_nombre                                               JUNTOS POR EL CAMBIO
jxc_id                                                                    132
jxc_votos                                                       

<h2>Chequear Totales</h2>

In [92]:
votos_positivos_totales = votos_considerados_flat["total_positivos"].sum()
votos_up = votos_considerados_flat["up_votos"].sum()
votos_fit = votos_considerados_flat["fit_votos"].sum()
votos_jxc = votos_considerados_flat["jxc_votos"].sum()
votos_lla = votos_considerados_flat["lla_votos"].sum()
votos_hxp = votos_considerados_flat["hxp_votos"].sum()
votos_nulos = votos_considerados_flat["nulo"].sum()
votos_blanco = votos_considerados_flat["en_blanco"].sum()

print(f"UP: {votos_up / votos_positivos_totales * 100} ({votos_up} votos)")
print(f"FIT: {votos_fit / votos_positivos_totales * 100} ({votos_fit} votos)")
print(f"JXC: {votos_jxc / votos_positivos_totales * 100} ({votos_jxc} votos)")
print(f"LLA: {votos_lla / votos_positivos_totales * 100} ({votos_lla} votos)")
print(f"HXP: {votos_hxp / votos_positivos_totales * 100} ({votos_hxp} votos)")
print(f"NULOS: {votos_nulos / votos_positivos_totales * 100} ({votos_nulos} votos)")
print(f"EN BLANCO: {votos_blanco / votos_positivos_totales * 100} ({votos_blanco} votos)")


UP: 36.688294770239054 (9645983 votos)
FIT: 2.7002115266868447 (709932 votos)
JXC: 23.836981668523904 (6267152 votos)
LLA: 29.987907218539313 (7884336 votos)
HXP: 6.7866048160108825 (1784315 votos)
NULOS: 0.8552655250600208 (224864 votos)
EN BLANCO: 2.107739783303624 (554161 votos)


<img src="totales_oficiales_2023.png">

<h2>EXPORTAR A CSV</h2>

In [94]:
votos_considerados_flat.to_csv('elecciones_2023_votos_flat.csv', index=False, encoding='latin1')